# AIMO Prize 3 - OOP Baseline Submission (v5)

This notebook demonstrates how to use the Object-Oriented Baseline with **Few-Shot Prompting**, **Majority Voting**, and **Parquet Submission**.

## Setup Instructions
1. **Add Utility Script**: Upload `src/kaggle_baseline.py` as a Dataset (e.g., named `aimo-pp3-source`).
2. **Add Model**: Search for and add the model `Qwen/Qwen2.5-Math-7B-Instruct`.
3. **Attach Competition Data**: Ensure the AIMO 3 competition data is attached.
4. **Run**: Execute the cells below.

In [ ]:
import os
import glob
import sys

print('--- DIAGNOSTIC INFO ---')
print(f'Current working directory: {os.getcwd()}')
print('Contents of /kaggle/input/:')
try:
    for item in sorted(os.listdir('/kaggle/input')):
        full_path = os.path.join('/kaggle/input', item)
        if os.path.isdir(full_path):
            print(f'  {item}/')
            if any(x in item.lower() for x in ['competition', 'prize', 'module', 'qwen']):
                try:
                    for sub_item in sorted(os.listdir(full_path)):
                        print(f'    - {sub_item}')
                except Exception as e:
                    print(f'    (Error listing {item}: {e})')
        else:
            print(f'  {item}')
except Exception as e:
    print(f'Error listing /kaggle/input/: {e}')

print('\nEnvironment variables:')
for key, value in sorted(os.environ.items()):
    if key.startswith('KAGGLE') or 'PATH' in key or 'HOME' in key:
        print(f'{key}={value}')

print('--- END DIAGNOSTIC INFO ---')


In [ ]:
import sys
import os
import glob

# 1. Import the OOP Baseline Module
sys.path.append('/kaggle/input/aimo-pp3-source')
sys.path.append(os.path.join(os.getcwd(), 'src')) 

try:
    from kaggle_baseline import CompetitionConfig, AIMSolver, MockLLM, VLLMEngine
    print("✅ Successfully imported OOP Baseline modules.")
except ImportError as e:
    print(f"❌ Import Failed: {e}. Make sure the 'aimo-pp3-source' dataset is attached.")

In [ ]:
# 2. Configure Environment & Model
config = CompetitionConfig()
config.n_repetitions = 16 

print(f"Environment: {'Kaggle' if config.is_kaggle else 'Local'}")

if config.is_kaggle and os.path.exists(config.model_path):
    print(f"Loading real VLLM Engine from {config.model_path}...")
    llm = VLLMEngine(config)
else:
    print("Using MockLLM for local testing or if model path not found.")
    llm = MockLLM()

solver = AIMSolver(config, llm)

## Submission Loop (Parquet)

In [ ]:
import sys
import os
import glob
import pandas as pd

# --------------------------------------------------------------------------------
# 3. AIMO 3 API Setup (Inference Server Pattern)
# --------------------------------------------------------------------------------
aimo_server_mod = None

api_files = glob.glob('/kaggle/input/**/aimo_3_inference_server.py', recursive=True)
if not api_files:
    api_files = glob.glob('data/**/aimo_3_inference_server.py', recursive=True)

if api_files:
    api_path = os.path.dirname(api_files[0])
    if os.path.basename(api_path) == 'kaggle_evaluation':
        parent_dir = os.path.dirname(api_path)
        if parent_dir not in sys.path: sys.path.append(parent_dir)
    elif api_path not in sys.path: sys.path.append(api_path)

    try:
        import aimo_3_inference_server as aimo_server_mod
        print('✅ Imported aimo_3_inference_server.')
    except ImportError:
        try:
            from kaggle_evaluation import aimo_3_inference_server as aimo_server_mod
            print('✅ Imported aimo_3_inference_server from package.')
        except ImportError as e:
            print(f'❌ Failed to import API: {e}')

found_test_csv = glob.glob('/kaggle/input/**/test.csv', recursive=True)
actual_test_csv = found_test_csv[0] if found_test_csv else '/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv'

def predict(*args, **kwargs):
    try:
        input_data = args[0] if args else None
        problem_text = 'What is 0+0?'
        if hasattr(input_data, 'columns') and 'problem' in input_data.columns:
            try:
                problem_text = str(input_data['problem'][0])
            except:
                problem_text = str(input_data.iloc[0]['problem'])
        elif isinstance(input_data, str):
            problem_text = input_data
        
        answer = solver.solve(problem_text)
        return pd.DataFrame({'answer': [answer]})
    except Exception as e:
        print(f'Predict Error: {e}')
        return pd.DataFrame({'answer': [0]})

if aimo_server_mod:
    try:
        import aimo_3_gateway
        old_gen = aimo_3_gateway.AIMO3Gateway.generate_data_batches
        def patched_gen(self):
            for data_batch, row_ids in old_gen(self):
                yield (data_batch,), row_ids
        aimo_3_gateway.AIMO3Gateway.generate_data_batches = patched_gen
        aimo_3_gateway.AIMO3Gateway.target_column_name = 'answer'
        aimo_3_gateway.AIMO3Gateway.row_id_column_name = 'id'
        print('✅ Applied Gateway monkey-patches.')
    except Exception as e:
        print(f'⚠️ Failed to patch Gateway: {e}')

    server = aimo_server_mod.AIMO3InferenceServer(predict)
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        server.serve()
    else:
        try:
            server.run_local_gateway(data_paths=(actual_test_csv,))
            print('✅ Local Gateway finished.')
        except Exception as e:
            print(f'❌ Local Gateway failed: {e}')
            pd.DataFrame({'id': ['test'], 'answer': [0]}).to_parquet('submission.parquet', index=False)
else:
    pd.DataFrame({'id': ['test'], 'answer': [0]}).to_parquet('submission.parquet', index=False)
